<a href="https://colab.research.google.com/github/belellei/data_enginner_study/blob/main/teste_tecnico_eng_dados_marina_bellei.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL

Com base nas tabelas: ori_lancamentos, ori_historico, ori_cooperativa e ori_canal, desenvolva exemplos de scripts para cada um dos itens abaixo, utilizando as tabelas fornecidas no item 1.1:

a. Where

b. Having

c. Like ou Not like

d. In ou Not in

e. And e Or

f. Exists ou Not Exists

g. Group By e Order by

h. Union

i. Intersect e Minus

j. Distinct

k. With

l. Sum, Min e Max

m. Update

n. Delete

In [471]:
# Importar bibliotecas
import csv
import pandas as pd
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import avg, col, sum, count, desc

In [472]:
# Inicializar uma sessão do Spark
spark = SparkSession.builder \
    .appName("SQL_Tables") \
    .getOrCreate()


In [473]:
# Listar arquivos na pasta atual
csv_files = [f for f in os.listdir('/content') if f.endswith('.csv')]
print(f"Arquivos csv encontrados: {csv_files}")

Arquivos csv encontrados: ['ori_historico.csv', 'ori_canal.csv', 'ori_cooperativa.csv', 'ori_lancamentos.csv']


In [474]:
# Definir o caminho dos arquivos
file_paths = {
    "ori_canal": "/content/ori_canal.csv",
    "ori_cooperativa": "/content/ori_cooperativa.csv",
    "ori_historico": "/content/ori_historico.csv",
    "ori_lancamentos": "/content/ori_lancamentos.csv"
}

In [475]:
# Função para detectar o separador automaticamente
def detect_separator(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            sample = file.readline()  # Lê a primeira linha do arquivo
            possible_separators = [",", ";", "\t", "|"]
            detected_separator = max(possible_separators, key=lambda sep: sample.count(sep))
            return detected_separator
    except Exception as e:
        return str(e)

# Dicionário para armazenar os DataFrames
dfs = {}

In [476]:
# Carregar os arquivos com o separador correto
for name, path in file_paths.items():
    detected_sep = detect_separator(path)  # Detectar o separador automaticamente

    if isinstance(detected_sep, str) and "Errno" in detected_sep:
        print(f"Erro ao detectar separador de {name}: {detected_sep}")
        continue

    try:
        df = pd.read_csv(path, header=0, sep=detected_sep, encoding="utf-8")  # Carregar CSV com o separador detectado
        dfs[name] = df
        print(f"Arquivo {name} carregado com sucesso! Separador detectado: '{detected_sep}'")
    except Exception as e:
        print(f"Erro ao carregar {name}: {e}")

Arquivo ori_canal carregado com sucesso! Separador detectado: ';'
Arquivo ori_cooperativa carregado com sucesso! Separador detectado: ';'
Arquivo ori_historico carregado com sucesso! Separador detectado: ';'
Arquivo ori_lancamentos carregado com sucesso! Separador detectado: ';'


In [477]:
# Converter os arquivos CSV diretamente para DataFrames Spark
spark_dfs = {}

for name, path in file_paths.items():
    try:
        # Detectar separador automaticamente
        detected_sep = detect_separator(path)

        # Carregar o CSV como DataFrame Spark
        spark_df = spark.read.csv(path, header=True, inferSchema=True, sep=detected_sep)

        # Armazenar no dicionário
        spark_dfs[name] = spark_df

        print(f"Tabela {name} carregada com sucesso!")
    except Exception as e:
        print(f"Erro ao carregar {name}: {e}")

Tabela ori_canal carregada com sucesso!
Tabela ori_cooperativa carregada com sucesso!
Tabela ori_historico carregada com sucesso!
Tabela ori_lancamentos carregada com sucesso!


In [478]:
# Exibir o esquema das tabelas no PySpark
for name, spark_df in spark_dfs.items():
    print(f"\nEsquema da tabela {name}:")
    spark_df.printSchema()


Esquema da tabela ori_canal:
root
 |-- CDCANAL: integer (nullable = true)
 |-- NMCANAL: string (nullable = true)


Esquema da tabela ori_cooperativa:
root
 |-- CDCOOPER: integer (nullable = true)
 |-- NMRESCOP: string (nullable = true)


Esquema da tabela ori_historico:
root
 |-- CDCOOPER: integer (nullable = true)
 |-- CDHISTOR: integer (nullable = true)
 |-- INDEBCRE: string (nullable = true)


Esquema da tabela ori_lancamentos:
root
 |-- DTMVTOLT: string (nullable = true)
 |-- CDCOOPER: integer (nullable = true)
 |-- CDHISTOR: integer (nullable = true)
 |-- CDCANAL: integer (nullable = true)
 |-- VLLANMTO: string (nullable = true)



In [479]:
# Converter os DataFrames pandas para PySpark e criar tabelas temporárias
for name, df in dfs.items():
    spark_df = spark.createDataFrame(df)
    spark_df.createOrReplaceTempView(name)
    spark_dfs[name] = spark_df
    print(f"Tabela {name} criada com sucesso!")

Tabela ori_canal criada com sucesso!
Tabela ori_cooperativa criada com sucesso!
Tabela ori_historico criada com sucesso!
Tabela ori_lancamentos criada com sucesso!


In [480]:
# Testar a consulta SQL
try:
    print("Exemplo de consulta na tabela 'ori_canal':")
    spark.sql("SELECT * FROM ori_canal LIMIT 10").show()
except Exception as e:
    print(f"Erro ao executar a consulta: {e}")

Exemplo de consulta na tabela 'ori_canal':
+-------+----------+
|CDCANAL|   NMCANAL|
+-------+----------+
|      1|    AIMARO|
|      2|     CAIXA|
|      3|  INTERNET|
|      4|       TAA|
|      5|AIMARO WEB|
|      6|       URA|
|      7|  PROCESSO|
|      8|MENSAGERIA|
|      9|   ESTEIRA|
|     10|    MOBILE|
+-------+----------+



**a. Where**

In [481]:
# Exibir o esquema (campos) das tabelas ori_lancamentos e ori_historico
print("Esquema da tabela ori_lancamentos:")
spark.sql("DESCRIBE ori_lancamentos").show(truncate=False)

print("Esquema da tabela ori_historico:")
spark.sql("DESCRIBE ori_historico").show(truncate=False)


Esquema da tabela ori_lancamentos:
+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|DTMVTOLT|string   |NULL   |
|CDCOOPER|bigint   |NULL   |
|CDHISTOR|bigint   |NULL   |
|CDCANAL |bigint   |NULL   |
|VLLANMTO|string   |NULL   |
+--------+---------+-------+

Esquema da tabela ori_historico:
+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|CDCOOPER|bigint   |NULL   |
|CDHISTOR|bigint   |NULL   |
|INDEBCRE|string   |NULL   |
+--------+---------+-------+



In [482]:
# 2. Exibir os 10 primeiros registros de cada tabela
print("Registros da tabela ori_lancamentos:")
spark.sql("SELECT * FROM ori_lancamentos LIMIT 10").show(truncate=False)

print("Registros da tabela ori_historico:")
spark.sql("SELECT * FROM ori_historico LIMIT 10").show(truncate=False)

Registros da tabela ori_lancamentos:
+----------+--------+--------+-------+--------+
|DTMVTOLT  |CDCOOPER|CDHISTOR|CDCANAL|VLLANMTO|
+----------+--------+--------+-------+--------+
|13/09/2021|1       |1       |0      |1,16    |
|13/09/2021|1       |1       |0      |2,68    |
|13/09/2021|1       |1       |0      |150     |
|13/09/2021|1       |1       |0      |200     |
|13/09/2021|1       |1       |0      |200     |
|13/09/2021|1       |1       |0      |234     |
|13/09/2021|1       |1       |0      |250     |
|13/09/2021|1       |1       |0      |272     |
|13/09/2021|1       |1       |0      |290     |
|13/09/2021|1       |1       |0      |300     |
+----------+--------+--------+-------+--------+

Registros da tabela ori_historico:
+--------+--------+--------+
|CDCOOPER|CDHISTOR|INDEBCRE|
+--------+--------+--------+
|1       |1       |C       |
|1       |2       |C       |
|1       |3       |C       |
|1       |4       |C       |
|1       |5       |C       |
|1       |6       |C   

In [483]:
# a. Where
consulta_sql = """
SELECT
    l.DTMVTOLT,
    l.CDCOOPER,
    l.CDHISTOR,
    l.CDCANAL,
    l.VLLANMTO,
    h.INDEBCRE
FROM ori_lancamentos AS l
INNER JOIN ori_historico AS h
    ON l.CDCOOPER = h.CDCOOPER AND l.CDHISTOR = h.CDHISTOR
WHERE l.VLLANMTO > 1000 -- Filtrar lançamentos acima de 1000
ORDER BY l.DTMVTOLT DESC
"""

# Executar a consulta no Spark SQL
print("Executando a consulta com WHERE:")
spark.sql(consulta_sql).show(truncate=False)

Executando a consulta com WHERE:
+----------+--------+--------+-------+--------+--------+
|DTMVTOLT  |CDCOOPER|CDHISTOR|CDCANAL|VLLANMTO|INDEBCRE|
+----------+--------+--------+-------+--------+--------+
|14/09/2021|3       |3338    |0      |3617    |C       |
|14/09/2021|3       |1006    |0      |4850    |C       |
|14/09/2021|1       |538     |0      |1700    |D       |
|14/09/2021|3       |802     |0      |6600    |C       |
|14/09/2021|3       |790     |0      |3296    |D       |
|14/09/2021|3       |795     |0      |72320   |D       |
|14/09/2021|3       |2227    |0      |4128    |D       |
|14/09/2021|5       |3320    |21     |1400    |D       |
|14/09/2021|7       |537     |0      |2060    |D       |
|14/09/2021|3       |1013    |0      |68517   |C       |
|14/09/2021|3       |1525    |0      |13565   |D       |
|14/09/2021|3       |1013    |0      |15600   |C       |
|14/09/2021|3       |1006    |0      |259165  |C       |
|14/09/2021|3       |1013    |0      |7000    |C       

**b. Having**

In [484]:
# b. Having

consulta_sql = """
SELECT
    l.CDCOOPER,
    l.CDHISTOR,
    SUM(l.VLLANMTO) AS TOTAL_LANCAMENTOS
FROM ori_lancamentos AS l
INNER JOIN ori_historico AS h
    ON l.CDCOOPER = h.CDCOOPER AND l.CDHISTOR = h.CDHISTOR
GROUP BY l.CDCOOPER, l.CDHISTOR
HAVING SUM(l.VLLANMTO) > 5000
ORDER BY TOTAL_LANCAMENTOS DESC
"""

# Executar a consulta
print(" Executando a consulta com HAVING:")
spark.sql(consulta_sql).show(truncate=False)


 Executando a consulta com HAVING:
+--------+--------+-----------------+
|CDCOOPER|CDHISTOR|TOTAL_LANCAMENTOS|
+--------+--------+-----------------+
|3       |784     |5317494.0        |
|3       |1006    |490683.0         |
|3       |1013    |232418.0         |
|3       |1005    |220916.0         |
|3       |797     |196406.0         |
|1       |539     |175248.0         |
|3       |1012    |124781.0         |
|3       |803     |102586.0         |
|3       |795     |72320.0          |
|1       |1       |65299.0          |
|3       |1527    |56712.0          |
|3       |3536    |52000.0          |
|1       |3318    |40563.0          |
|1       |537     |34489.0          |
|1       |508     |34413.0          |
|3       |885     |23130.0          |
|1       |316     |21030.0          |
|3       |3368    |20404.0          |
|1       |2433    |18331.0          |
|3       |574     |18120.0          |
+--------+--------+-----------------+
only showing top 20 rows



**c. Like**

*usei like por ser uma boa prática, isso porque irá buscar as referencias relacionadas a condição. NOT LIKE precisa buscar todas as não referencias, para entao buscar as referencias relacionadas a condição.*

In [485]:
# c. Like

consulta_sql = """
SELECT
    CDCOOPER,
    CDHISTOR,
    VLLANMTO
FROM ori_lancamentos
WHERE CDHISTOR LIKE '1%' -- Filtra históricos que começam com '1'
ORDER BY VLLANMTO DESC
"""

# Executar a consulta
print("Executando a consulta com LIKE:")
spark.sql(consulta_sql).show(truncate=False)


Executando a consulta com LIKE:
+--------+--------+--------+
|CDCOOPER|CDHISTOR|VLLANMTO|
+--------+--------+--------+
|1       |1545    |991,25  |
|1       |1548    |99,54   |
|3       |1005    |9880    |
|1       |15      |9800    |
|16      |1548    |980     |
|3       |1005    |97741   |
|1       |1545    |96,43   |
|1       |1548    |96,37   |
|3       |1005    |9485    |
|3       |1006    |9485    |
|3       |1013    |94503   |
|1       |1       |925     |
|3       |1012    |90134,83|
|1       |1       |900     |
|1       |1       |900     |
|1       |127     |90      |
|1       |1548    |90      |
|1       |1548    |9,99    |
|1       |1548    |9,94    |
|1       |1548    |9,8     |
+--------+--------+--------+
only showing top 20 rows



**d. IN**

*usei IN com a mesma justificativa de usar LIKE*

In [486]:
# d. In

consulta_sql = """
SELECT
    CDCOOPER,
    CDHISTOR,
    CDCANAL,
    VLLANMTO
FROM ori_lancamentos
WHERE CDCOOPER IN (1, 2, 3) -- Filtra apenas essas cooperativas
ORDER BY VLLANMTO DESC
"""

# Executar a consulta
print("Executando a consulta com IN:")
spark.sql(consulta_sql).show(truncate=False)


Executando a consulta com IN:
+--------+--------+-------+---------+
|CDCOOPER|CDHISTOR|CDCANAL|VLLANMTO |
+--------+--------+-------+---------+
|1       |987     |0      |999,73   |
|3       |440     |0      |9915,6   |
|1       |1545    |0      |991,25   |
|1       |508     |0      |99,9     |
|1       |1548    |8      |99,54    |
|3       |1005    |0      |9880     |
|1       |15      |3      |9800     |
|1       |3318    |21     |980      |
|1       |2741    |0      |98,55    |
|3       |1005    |0      |97741    |
|1       |987     |0      |9772,96  |
|1       |3318    |21     |976      |
|3       |791     |0      |97139,56 |
|1       |856     |0      |97,41    |
|3       |2623    |0      |9685,71  |
|1       |3373    |21     |966,74   |
|3       |3044    |0      |965267,14|
|3       |797     |0      |96293,51 |
|1       |1545    |0      |96,43    |
|1       |1548    |8      |96,37    |
+--------+--------+-------+---------+
only showing top 20 rows



**e. And e Or**

In [487]:
# e. And e Or

consulta_sql = """
SELECT
    CDCOOPER,
    CDHISTOR,
    CDCANAL,
    VLLANMTO
FROM ori_lancamentos
WHERE (CDCOOPER = 1 OR CDCOOPER = 2)  -- Condição OR
AND VLLANMTO > 5000  -- Condição AND
ORDER BY VLLANMTO DESC
"""

# Executar a consulta
print("Executando a consulta com AND e OR:")
spark.sql(consulta_sql).show(truncate=False)


Executando a consulta com AND e OR:
+--------+--------+-------+--------+
|CDCOOPER|CDHISTOR|CDCANAL|VLLANMTO|
+--------+--------+-------+--------+
|1       |15      |3      |9800    |
|1       |578     |0      |9000    |
|1       |1       |0      |7000    |
|1       |1       |0      |7000    |
|1       |539     |0      |7000    |
|1       |1030    |0      |6000    |
|1       |539     |0      |5475    |
|1       |508     |0      |5390    |
|1       |1       |0      |22647   |
|1       |539     |0      |16500   |
|1       |2433    |0      |13384   |
|1       |386     |0      |12634   |
|1       |539     |0      |120000  |
+--------+--------+-------+--------+



**f. Exists**

*mesma justifica de utilizar IN ao invés de NOT IN*

In [488]:
# f. Exists

consulta_sql = """
SELECT
    l.CDCOOPER,
    l.CDHISTOR,
    l.CDCANAL,
    l.VLLANMTO
FROM ori_lancamentos AS l
WHERE EXISTS (
    SELECT 1
    FROM ori_historico AS h
    WHERE l.CDCOOPER = h.CDCOOPER
    AND l.CDHISTOR = h.CDHISTOR
)
ORDER BY l.VLLANMTO DESC
LIMIT 10
"""

# Executar a consulta
print("Executando a consulta com EXISTS:")
spark.sql(consulta_sql).show(truncate=False)


Executando a consulta com EXISTS:
+--------+--------+-------+--------+
|CDCOOPER|CDHISTOR|CDCANAL|VLLANMTO|
+--------+--------+-------+--------+
|1       |987     |0      |999,73  |
|3       |440     |0      |9915,6  |
|1       |1545    |0      |991,25  |
|7       |508     |0      |99,9    |
|1       |508     |0      |99,9    |
|1       |1548    |8      |99,54   |
|3       |1005    |0      |9880    |
|1       |15      |3      |9800    |
|1       |3318    |21     |980     |
|16      |1548    |8      |980     |
+--------+--------+-------+--------+



**g. Group By e Order by**

In [489]:
# g. Group By e Order by

consulta_sql = """
SELECT
    CDCOOPER,
    CDHISTOR,
    SUM(VLLANMTO) AS TOTAL_LANCAMENTOS
FROM ori_lancamentos
GROUP BY CDCOOPER, CDHISTOR  -- Agrupa por cooperativa e histórico
ORDER BY TOTAL_LANCAMENTOS DESC  -- Ordena do maior para o menor
"""

# Executar a consulta
print("Executando a consulta com GROUP BY e ORDER BY:")
spark.sql(consulta_sql).show(truncate=False)


Executando a consulta com GROUP BY e ORDER BY:
+--------+--------+-----------------+
|CDCOOPER|CDHISTOR|TOTAL_LANCAMENTOS|
+--------+--------+-----------------+
|3       |784     |5317494.0        |
|3       |1006    |490683.0         |
|3       |1013    |232418.0         |
|3       |1005    |220916.0         |
|3       |797     |196406.0         |
|1       |539     |175248.0         |
|3       |1012    |124781.0         |
|3       |803     |102586.0         |
|3       |795     |72320.0          |
|1       |1       |65299.0          |
|3       |1527    |56712.0          |
|3       |3536    |52000.0          |
|1       |3318    |40563.0          |
|1       |537     |34489.0          |
|1       |508     |34413.0          |
|3       |885     |23130.0          |
|1       |316     |21030.0          |
|3       |3368    |20404.0          |
|1       |2433    |18331.0          |
|3       |574     |18120.0          |
+--------+--------+-----------------+
only showing top 20 rows



**h. Union**

In [490]:
# h. Union

consulta_sql = """
SELECT CDCOOPER, CDHISTOR, CDCANAL, VLLANMTO
FROM ori_lancamentos
WHERE CDCOOPER IN (1001, 1002)

UNION

SELECT CDCOOPER, CDHISTOR, CDCANAL, VLLANMTO
FROM ori_lancamentos
WHERE CDCOOPER IN (3, 4)

ORDER BY VLLANMTO DESC
"""

# Executar a consulta
print("Executando a consulta com UNION:")
spark.sql(consulta_sql).show(truncate=False)



Executando a consulta com UNION:
+--------+--------+-------+----------+
|CDCOOPER|CDHISTOR|CDCANAL|VLLANMTO  |
+--------+--------+-------+----------+
|3       |440     |0      |9915,6    |
|3       |1005    |0      |9880      |
|3       |1005    |0      |97741     |
|3       |791     |0      |97139,56  |
|3       |2623    |0      |9685,71   |
|3       |3044    |0      |965267,14 |
|3       |797     |0      |96293,51  |
|3       |3374    |0      |9490922,77|
|3       |1005    |0      |9485      |
|3       |1006    |0      |9485      |
|3       |1013    |0      |94503     |
|3       |3368    |0      |92865,85  |
|3       |2227    |0      |926,4     |
|3       |3372    |0      |9231215,5 |
|3       |2643    |0      |91,14     |
|3       |1012    |0      |90134,83  |
|3       |796     |0      |88743,76  |
|3       |797     |0      |8837514,42|
|3       |885     |0      |87966,5   |
|3       |2239    |0      |8695,66   |
+--------+--------+-------+----------+
only showing top 20 rows



**i. Intersect e Minus**

In [491]:
# i. Intersect

consulta_sql = """
SELECT CDCOOPER, CDHISTOR
FROM ori_lancamentos

INTERSECT

SELECT CDCOOPER, CDHISTOR
FROM ori_historico

ORDER BY CDCOOPER, CDHISTOR
"""

# Executar a consulta
print("Executando a consulta com INTERSECT:")
spark.sql(consulta_sql).show(truncate=False)



Executando a consulta com INTERSECT:
+--------+--------+
|CDCOOPER|CDHISTOR|
+--------+--------+
|1       |1       |
|1       |15      |
|1       |21      |
|1       |22      |
|1       |47      |
|1       |48      |
|1       |108     |
|1       |127     |
|1       |316     |
|1       |386     |
|1       |508     |
|1       |521     |
|1       |537     |
|1       |538     |
|1       |539     |
|1       |555     |
|1       |578     |
|1       |667     |
|1       |834     |
|1       |856     |
+--------+--------+
only showing top 20 rows



In [492]:
# i. Minus

consulta_sql = """
SELECT CDCOOPER, CDHISTOR
FROM ori_lancamentos

EXCEPT

SELECT CDCOOPER, CDHISTOR
FROM ori_historico

ORDER BY CDCOOPER, CDHISTOR
"""

# Executar a consulta
print("Executando a consulta com MINUS (EXCEPT no PySpark):")
spark.sql(consulta_sql).show(truncate=False)


Executando a consulta com MINUS (EXCEPT no PySpark):
+--------+--------+
|CDCOOPER|CDHISTOR|
+--------+--------+
+--------+--------+



**j. Distinct**

In [493]:
# j. Distinct

consulta_sql = """
SELECT DISTINCT CDCOOPER, CDHISTOR
FROM ori_lancamentos
ORDER BY CDCOOPER, CDHISTOR
"""

# Executar a consulta
print("Executando a consulta com DISTINCT:")
spark.sql(consulta_sql).show(truncate=False)

Executando a consulta com DISTINCT:
+--------+--------+
|CDCOOPER|CDHISTOR|
+--------+--------+
|1       |1       |
|1       |15      |
|1       |21      |
|1       |22      |
|1       |47      |
|1       |48      |
|1       |108     |
|1       |127     |
|1       |316     |
|1       |386     |
|1       |508     |
|1       |521     |
|1       |537     |
|1       |538     |
|1       |539     |
|1       |555     |
|1       |578     |
|1       |667     |
|1       |834     |
|1       |856     |
+--------+--------+
only showing top 20 rows



**k. With**

In [494]:
# k. With

consulta_sql = """
WITH Lancamentos_Agrupados AS (
    SELECT
        CDCOOPER,
        CDHISTOR,
        SUM(VLLANMTO) AS TOTAL_LANCAMENTOS
    FROM ori_lancamentos
    GROUP BY CDCOOPER, CDHISTOR
)

SELECT *
FROM Lancamentos_Agrupados
WHERE TOTAL_LANCAMENTOS > 5000
ORDER BY TOTAL_LANCAMENTOS DESC
"""

# Executar a consulta
print("Executando a consulta com WITH (CTE):")
spark.sql(consulta_sql).show(truncate=False)



Executando a consulta com WITH (CTE):
+--------+--------+-----------------+
|CDCOOPER|CDHISTOR|TOTAL_LANCAMENTOS|
+--------+--------+-----------------+
|3       |784     |5317494.0        |
|3       |1006    |490683.0         |
|3       |1013    |232418.0         |
|3       |1005    |220916.0         |
|3       |797     |196406.0         |
|1       |539     |175248.0         |
|3       |1012    |124781.0         |
|3       |803     |102586.0         |
|3       |795     |72320.0          |
|1       |1       |65299.0          |
|3       |1527    |56712.0          |
|3       |3536    |52000.0          |
|1       |3318    |40563.0          |
|1       |537     |34489.0          |
|1       |508     |34413.0          |
|3       |885     |23130.0          |
|1       |316     |21030.0          |
|3       |3368    |20404.0          |
|1       |2433    |18331.0          |
|3       |574     |18120.0          |
+--------+--------+-----------------+
only showing top 20 rows



**l. Sum, Min e Max**

In [495]:
# l. Sum, Min e Max

consulta_sql = """
SELECT
    SUM(VLLANMTO) AS TOTAL_LANCAMENTOS,
    MIN(VLLANMTO) AS MENOR_LANCAMENTO,
    MAX(VLLANMTO) AS MAIOR_LANCAMENTO
FROM ori_lancamentos
GROUP BY CDCOOPER
ORDER BY TOTAL_LANCAMENTOS DESC
"""

# Executar a consulta
print("Executando a consulta com SUM, MIN e MAX:")
spark.sql(consulta_sql).show(truncate=False)

Executando a consulta com SUM, MIN e MAX:
+-----------------+----------------+----------------+
|TOTAL_LANCAMENTOS|MENOR_LANCAMENTO|MAIOR_LANCAMENTO|
+-----------------+----------------+----------------+
|6972669.0        |0,4             |9915,6          |
|503692.0         |0,01            |999,73          |
|25320.0          |1,94            |9,7             |
|9665.0           |10921,14        |888,3           |
|5407.0           |0,02            |68              |
|4123.0           |0,12            |980             |
|2659.0           |10              |99,9            |
|308.0            |110             |50              |
|150.0            |10              |9,48            |
|115.0            |115             |32,76           |
|20.0             |10              |88,18           |
|NULL             |371,16          |371,16          |
+-----------------+----------------+----------------+



**m. Update**

In [496]:
#  Update

consulta_sql = """
SELECT
    CDCOOPER,
    CDHISTOR,
    CDCANAL,
    CASE
        WHEN VLLANMTO < 1000 THEN VLLANMTO * 1.10  -- Aumenta em 10% os valores abaixo de 1000
        ELSE VLLANMTO  -- Mantém os outros valores iguais
    END AS VLLANMTO_ATUALIZADO
FROM ori_lancamentos
"""

# Executar a consulta
print("Simulando um UPDATE com CASE WHEN:")
spark.sql(consulta_sql).show(truncate=False)


Simulando um UPDATE com CASE WHEN:
+--------+--------+-------+-------------------+
|CDCOOPER|CDHISTOR|CDCANAL|VLLANMTO_ATUALIZADO|
+--------+--------+-------+-------------------+
|1       |1       |0      |1,16               |
|1       |1       |0      |2,68               |
|1       |1       |0      |165.0              |
|1       |1       |0      |220.00000000000003 |
|1       |1       |0      |220.00000000000003 |
|1       |1       |0      |257.40000000000003 |
|1       |1       |0      |275.0              |
|1       |1       |0      |299.20000000000005 |
|1       |1       |0      |319.0              |
|1       |1       |0      |330.0              |
|1       |1       |0      |374.00000000000006 |
|1       |1       |0      |385.00000000000006 |
|1       |1       |0      |550.0              |
|1       |1       |0      |660.0              |
|1       |1       |0      |660.0              |
|1       |1       |0      |770.0000000000001  |
|1       |1       |0      |770.0000000000001  |
|1   

In [497]:
# Criar uma nova tabela temporária com os valores atualizados (spark não permite update e delete direto, então nesse caso cria uma tabela nova e sobrescreve a original)
#df_atualizado = spark.sql(consulta_sql)

# Sobrescrever a tabela original
#df_atualizado.createOrReplaceTempView("ori_lancamentos")


In [498]:
# Update usando Tabelas Delta

#UPDATE ori_lancamentos
#SET VLLANMTO = VLLANMTO * 1.10
#WHERE VLLANMTO < 1000;



**n. Delete**

In [499]:
# n. Delete

consulta_sql = """
SELECT *
FROM ori_lancamentos
WHERE VLLANMTO >= 100  -- Mantém apenas valores iguais ou superiores a 100
"""

# Executar a consulta no Spark SQL
print("Simulando um DELETE com WHERE:")
spark.sql(consulta_sql).show(truncate=False)


Simulando um DELETE com WHERE:
+----------+--------+--------+-------+--------+
|DTMVTOLT  |CDCOOPER|CDHISTOR|CDCANAL|VLLANMTO|
+----------+--------+--------+-------+--------+
|13/09/2021|1       |1       |0      |150     |
|13/09/2021|1       |1       |0      |200     |
|13/09/2021|1       |1       |0      |200     |
|13/09/2021|1       |1       |0      |234     |
|13/09/2021|1       |1       |0      |250     |
|13/09/2021|1       |1       |0      |272     |
|13/09/2021|1       |1       |0      |290     |
|13/09/2021|1       |1       |0      |300     |
|13/09/2021|1       |1       |0      |340     |
|13/09/2021|1       |1       |0      |350     |
|13/09/2021|1       |1       |0      |500     |
|13/09/2021|1       |1       |0      |600     |
|13/09/2021|1       |1       |0      |600     |
|13/09/2021|1       |1       |0      |700     |
|13/09/2021|1       |1       |0      |700     |
|13/09/2021|1       |1       |0      |800     |
|13/09/2021|1       |1       |0      |850     |
|13/09/20

In [500]:
# Criar um novo DataFrame sem os registros indesejados (spark não permite update e delete direto, então nesse caso cria uma tabela nova e sobrescreve a original)
#df_filtrado = spark.sql(consulta_sql)

# Sobrescrever a tabela original (substituindo os dados)
#df_filtrado.createOrReplaceTempView("ori_lancamentos")

In [501]:
# Usando Delete no SQL

#DELETE FROM ori_lancamentos
#WHERE VLLANMTO < 100;


 **Escreva
uma consulta SELECT que retorne a média dos valores de lançamentos, agrupados
pelos seguintes campos: cooperativa, dtbase e dshistorico**

In [502]:
# Consulta Select

consulta_sql = """
SELECT
    l.CDCOOPER,
    l.DTMVTOLT AS DTBASE,
    h.INDEBCRE,  -- Usando um campo existente na tabela ori_historico
    AVG(l.VLLANMTO) AS MEDIA_LANCAMENTOS
FROM ori_lancamentos AS l
INNER JOIN ori_historico AS h
    ON l.CDCOOPER = h.CDCOOPER
    AND l.CDHISTOR = h.CDHISTOR
GROUP BY l.CDCOOPER, l.DTMVTOLT, h.INDEBCRE
ORDER BY l.CDCOOPER, DTBASE, MEDIA_LANCAMENTOS DESC
"""

# Executar a consulta
print("Executando a consulta:")
spark.sql(consulta_sql).show(truncate=False)



Executando a consulta:
+--------+----------+--------+------------------+
|CDCOOPER|DTBASE    |INDEBCRE|MEDIA_LANCAMENTOS |
+--------+----------+--------+------------------+
|1       |13/09/2021|C       |789.2087912087912 |
|1       |13/09/2021|D       |361.8764940239044 |
|1       |14/09/2021|C       |3877.5            |
|1       |14/09/2021|D       |441.39830508474574|
|2       |13/09/2021|C       |51.333333333333336|
|3       |14/09/2021|D       |98392.55932203389 |
|3       |14/09/2021|C       |17961.66153846154 |
|5       |14/09/2021|D       |525.2             |
|5       |14/09/2021|C       |77.5              |
|7       |13/09/2021|D       |40.0              |
|7       |14/09/2021|D       |624.75            |
|9       |14/09/2021|D       |37.5              |
|10      |13/09/2021|D       |10.0              |
|10      |13/09/2021|C       |NULL              |
|11      |13/09/2021|C       |1790.0            |
|11      |13/09/2021|D       |7.5               |
|11      |14/09/2021|D     

# Python

Escreva uma função Python que recebe duas strings no formato "YYYY-MM-DD",
converte para datas e retorna a diferença em dias entre essas datas.

Teste sua função usando as seguintes datas: "2022-01-01" e "2022-03-15".
Imprima o resultado na forma de uma mensagem usando f-string, com o seguinte
texto: "A diferença em dias entre as datas é: X" (onde X é a diferença calculada).

In [503]:
# Diferença entre datas
def diferenca_em_dias(data1: str, data2: str) -> int:
    return abs((datetime.strptime(data2, "%Y-%m-%d") - datetime.strptime(data1, "%Y-%m-%d")).days)

# Teste
print(f"A diferença entre as datas é: {diferenca_em_dias('2022-01-01', '2022-03-15')} dias")



A diferença entre as datas é: 73 dias


Crie um array chamado 'numeros' contendo os números de 1 a 5 e imprima o
array.
Depois imprima apenas os números ímpares do array, no final adicione o
número 6 no final do array 'numeros'.

In [504]:
# Criando o array com números de 1 a 5
numeros = [1, 2, 3, 4, 5]

# Imprimindo o array completo
print("Array completo:", numeros)

Array completo: [1, 2, 3, 4, 5]


In [505]:
# Filtrando apenas os números ímpares
numeros_impares = [num for num in numeros if num % 2 != 0]
print("Números ímpares:", numeros_impares)

Números ímpares: [1, 3, 5]


In [506]:
# Adicionando o número 6 ao final do array
numeros.append(6)

In [507]:
# Imprimindo o array atualizado
print("Array atualizado:", numeros)

Array atualizado: [1, 2, 3, 4, 5, 6]


Considere a tabela abaixo:

Nome Idade Nota

Maria 12 8,5

José 11 7,3

Aline 12 9,8

a) Monte o comando em pyspark que cria um dataframe com as seguintes
colunas: "nome" (string), "idade" (integer) e "nota" (double).

b) Informe o comando pyspark que executa uma consulta para selecionar
apenas as colunas "nome" e "nota" do dataframe.

a)

In [508]:
# Definir o esquema do DataFrame
schema = StructType([
    StructField("nome", StringType(), True),
    StructField("idade", IntegerType(), True),
    StructField("nota", DoubleType(), True)
])

In [509]:
# Criar os dados
dados = [("Maria", 12, 8.5), ("José", 11, 7.3), ("Aline", 12, 9.8)]

In [510]:
# Criar o DataFrame
df = spark.createDataFrame(dados, schema=schema)

In [511]:
# Exibir o DataFrame
df.show()

+-----+-----+----+
| nome|idade|nota|
+-----+-----+----+
|Maria|   12| 8.5|
| José|   11| 7.3|
|Aline|   12| 9.8|
+-----+-----+----+



b)

In [512]:
df.select("nome", "nota").show()

+-----+----+
| nome|nota|
+-----+----+
|Maria| 8.5|
| José| 7.3|
|Aline| 9.8|
+-----+----+

